In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import time

from tqdm import tqdm
from collections import defaultdict

from typing import List, Set

In [2]:
# generate all sequences of length l
def generate(l):
    res = []
    
    for i in range(0, 2**l):
        b = bin(i)[2:]
        b = '0' * (l - len(b)) + b
        b = np.array(list(map(lambda x: int(x), b)))
        res.append(b)
        
    return np.array(res)

In [3]:
n = 10
k = 6
r = 4
H = np.array([
    [0, 0, 0, 1, 1, 1, 1, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 0, 0]
])

G = np.array([
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 1, 0], 
        [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 1, 0]
])

# Минимальная спэновая форма
G = np.array([
        [1, 0, 0, 1, 1, 1, 1, 0, 0, 0],
        [0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], 
        [0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
])


p0 = 1e-5

all_words = generate(n)

code_words = []
for word in all_words:
    if np.all(np.dot(word, H.T) % 2 == 0):
        code_words.append(word)

In [4]:
np.dot(G, H.T) % 2

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

# Задание 4

## Построение по проверочной матрице (полная решетка в отдельном файле)

In [6]:
nodes = {() : [0,0,0,0]}

def is_code_word_prefix(code_words, check_word):
    size = len(check_word)
    for word in code_words:
        if (np.all(np.array(word[:size]) == np.array(check_word))):
            return True
    return False

def process_word(word_p, word_c, next_line):
    pref = np.dot(word_c, H[:, :i+1].T) % 2
    if is_code_word_prefix(code_words, word_c):
        next_line.append(word_c)
        if not (tuple(pref) in profile[len(word_c)]):
            profile[len(word_c)].add(tuple(pref))
        nodes[tuple(word_c)] = pref
        print('Node {} -> Node {}. Current sequence: {}'.format(nodes[tuple(word_p)], pref, word_c))

prev_line = [[]]
profile = []
for i in range(0, n + 1):
    profile.append(set())

for i in range(0, n):
    print('Level ', i)
    next_line = []
    for word in prev_line:
        process_word(word, [*word, 0], next_line)
        process_word(word, [*word, 1], next_line)
    prev_line = next_line

num_profile = []
for p in profile:
    num_profile.append(len(p))
num_profile[0] = 1

Level  0
Node [0, 0, 0, 0] -> Node [0 0 0 0]. Current sequence: [0]
Node [0, 0, 0, 0] -> Node [0 1 1 1]. Current sequence: [1]
Level  1
Node [0 0 0 0] -> Node [0 0 0 0]. Current sequence: [0, 0]
Node [0 0 0 0] -> Node [0 0 1 0]. Current sequence: [0, 1]
Node [0 1 1 1] -> Node [0 1 1 1]. Current sequence: [1, 0]
Node [0 1 1 1] -> Node [0 1 0 1]. Current sequence: [1, 1]
Level  2
Node [0 0 0 0] -> Node [0 0 0 0]. Current sequence: [0, 0, 0]
Node [0 0 0 0] -> Node [0 0 1 1]. Current sequence: [0, 0, 1]
Node [0 0 1 0] -> Node [0 0 1 0]. Current sequence: [0, 1, 0]
Node [0 0 1 0] -> Node [0 0 0 1]. Current sequence: [0, 1, 1]
Node [0 1 1 1] -> Node [0 1 1 1]. Current sequence: [1, 0, 0]
Node [0 1 1 1] -> Node [0 1 0 0]. Current sequence: [1, 0, 1]
Node [0 1 0 1] -> Node [0 1 0 1]. Current sequence: [1, 1, 0]
Node [0 1 0 1] -> Node [0 1 1 0]. Current sequence: [1, 1, 1]
Level  3
Node [0 0 0 0] -> Node [0 0 0 0]. Current sequence: [0, 0, 0, 0]
Node [0 0 0 0] -> Node [1 0 0 1]. Current sequenc

In [7]:
print('Профиль:', num_profile)

Профиль: [1, 2, 4, 8, 16, 16, 8, 8, 4, 2, 1]


## Построение по порождающей матрице

Профиль: (1 2 4 8 16 16 8 8 4 2 1)